In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'f9bbcd2d-0f8d-4678-9781-10e5463bae3b'}], 'layout': {'template': '...'}
})

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

<SocketContext(connect='tcp://0.0.0.0:5555')>

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

# RUN python TickServer.py in background

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

# Above graph updates in real time!

# SMA 3 Times

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': 'c033dd16-55ed-49d9-a0d3-87a4369e2181'},
             {'line': {'dash': 'dot', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA1',
              'type': 'scatter',
              'uid': '02f5185b-ab30-48c7-9f1f-18e6914add1a'},
             {'line': {'dash': 'dash', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA2',
              'type': 'scatter',
              'uid': 'fc566fcf-5715-4db0-b268-fc979d500466'}],
    'layout': {'template': '...'}
})

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '29fdfb0d-f59c-4b5b-9441-bac515111fa6',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'dash': 'dot', 'width': 1},
              'marker': {'symbol': 'triangle-up'},
              'mode': 'lines+markers',
              'name': 'RETURN',
              'type': 'scatter',
              'uid': '5bd85033-a692-479c-b102-103b7ce4def3',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'dash': 'dash', 'width': 1},
              'marker': {'symbol': 'x'},
              'mode': 'lines+markers',
              'name': 'MOMENTUM',
              'type': 'scatter',
              'uid': '690022f8-456e-47ce-944d-c581687ed5e8',
              'xaxis': 'x3',
              'yaxis': 'y3'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
               'x

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [31]:
socket = context.socket(zmq.SUB)

In [32]:
socket.connect('tcp://0.0.0.0:5556')

<SocketContext(connect='tcp://0.0.0.0:5556')>

In [33]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [34]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

KeyboardInterrupt: 

In [ ]:
fig = go.FigureWidget()
fig.add_bar()
fig

In [ ]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y